<a href="https://colab.research.google.com/github/JavierVillarAsensio/MLT-NaturalLanguageProcessing/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
drive.mount('/content/drive')
df_prods = pd.read_csv("drive/MyDrive/products.csv", on_bad_lines='skip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Id                              object
ProductId                       object
UserId                          object
ProfileName                     object
HelpfulnessNumerator            object
HelpfulnessDenominator          object
Score                           object
Time                            object
Summary                         object
Text;;;;;;;;;;;;;;;;;;;;;;;;    object
dtype: object

In [ ]:
#There are blank columns after Text, that is the reason of the ;
texts = df_prods["Text;;;;;;;;;;;;;;;;;;;;;;;;"]
summs = df_prods["Summary"]
d = {'Text': texts, 'Summary': summs}
texts_summs_df = pd.DataFrame(data=d)


In [ ]:
bad_chars = ["!", "\"" "_",  "$", "%", "&", "/", "(", ")", "=", "_", "ˆ", "*", "¡", "@"]
texts_summs_df.replace(bad_chars)
texts_summs_df['Text'].str.lower()
texts_summs_df['Summary'].str.lower()

0         good quality dog food
1                           NaN
2                           NaN
3                cough medicine
4                           NaN
                  ...          
566788                      NaN
566789                      NaN
566790                      NaN
566791                      NaN
566792                      NaN
Name: Summary, Length: 566793, dtype: object